## ENV

In [25]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installation

In [37]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# Dataset Prep

In [27]:
# Data loading
import os
import json
import pandas as pd

burnfit_dir = os.path.join("/content/drive/MyDrive", "Burnfit")
input_dir = os.path.join(burnfit_dir, "input")
output_dir = os.path.join(burnfit_dir, "output")

inputs = os.listdir(input_dir)
outputs = os.listdir(output_dir)

input_user_profile_csv = os.path.join(input_dir, "input_user.csv")
input_user_1rm_csv = os.path.join(input_dir, "input_user_rm.csv")
output_increase_rate_csv = os.path.join(output_dir, "output_increase_rate.csv")
output_weekly_increase_rate_plan_csv = os.path.join(output_dir, "output_weekly_increase_rate_plan.csv")

input_user_profile_df = pd.read_csv(input_user_profile_csv)
input_user_1rm_df = pd.read_csv(input_user_1rm_csv)
output_increase_rate_df = pd.read_csv(output_increase_rate_csv)
output_weekly_increase_rate_plan_df = pd.read_csv(output_weekly_increase_rate_plan_csv)

print(input_user_profile_df.columns)
print(input_user_1rm_df.columns)
print(output_increase_rate_df.columns)
print(output_weekly_increase_rate_plan_df.columns)

Index(['id', '운동경험', '성별', '운동목표'], dtype='object')
Index(['id', '벤치프레스', '스쿼트', '데드리프트', '오버헤드프레스'], dtype='object')
Index(['id', 'init_weight_rate', 'increase_rate_week', 'increase_rate_set',
       'deloading_rate'],
      dtype='object')
Index(['id', 'weekly_increase_rate_plan'], dtype='object')


In [28]:
# 질문
def question_formmatter(input_row):
  input_text = [
        "## 질문",
        "-5/3/1 프로그램",
        f"- **운동경험**: {input_row['운동경험']}",
        f"- **성별**: {input_row['성별']}",
        f"- **운동목표**: {input_row['운동목표']}",
        "- **1RM**:",
        f"  - 벤치프레스: {input_row['벤치프레스']}",
        f"  - 스쿼트: {input_row['스쿼트']}",
        f"  - 데드리프트: {input_row['데드리프트']}",
        f"  - 오버헤드프레스: {input_row['오버헤드프레스']}"
    ]
  return "\n".join(input_text)

# 답변
def answer_formmatter(output_row):
  json_output = {
        "program": "5/3/1 프로그램",
        "init_weight_rate": str(output_row["init_weight_rate"]),
        "increase_rate_week": str(output_row["increase_rate_week"]),
        "increase_rate_set": str(output_row["increase_rate_set"]),
        "deloading_rate": str(output_row["deloading_rate"]),
        "weekly_weight_increase_plan": output_row["weekly_increase_rate_plan"]
  }
  # answer = f"## 답변\n{json.dumps(answer_json, ensure_ascii=False, indent=2)}\n<END>"
  return f"## 답변\n{json.dumps(json_output, ensure_ascii=False, indent=2)}"


## 질문 + 답변
def question_answer_formmatter(input_user_profile_df, input_user_1rm_df, output_increase_rate_df, output_weekly_increase_rate_plan_df):
    rows = []
    combined_df = input_user_profile_df.merge(input_user_1rm_df, on="id") \
            .merge(output_increase_rate_df, on="id") \
            .merge(output_weekly_increase_rate_plan_df, on="id")

    for _, row in combined_df.iterrows():
        input_row = row[["운동경험", "성별", "운동목표", "벤치프레스", "스쿼트", "데드리프트", "오버헤드프레스"]]
        output_row = row[["init_weight_rate", "increase_rate_week", "increase_rate_set", "deloading_rate", "weekly_increase_rate_plan"]]

        question = question_formmatter(input_row)
        answer = answer_formmatter(output_row)
        full_text = f"{question}\n\n{answer}"
        rows.append({"id": row["id"], "text": full_text})
        if _ < 5:
          print(full_text)
    final_df = pd.DataFrame(rows)
    return final_df

In [29]:
text_df = question_answer_formmatter(input_user_profile_df, input_user_1rm_df, output_increase_rate_df, output_weekly_increase_rate_plan_df)
print("Rows:", text_df.shape[0])      # Should be 1000
print("Columns:", text_df.shape[1])

## 질문
-5/3/1 프로그램
- **운동경험**: 1
- **성별**: 남성
- **운동목표**: 4
- **1RM**:
  - 벤치프레스: 30kg
  - 스쿼트: 30kg
  - 데드리프트: 30kg
  - 오버헤드프레스: 30kg

## 답변
{
  "program": "5/3/1 프로그램",
  "init_weight_rate": "45",
  "increase_rate_week": "10",
  "increase_rate_set": "5",
  "deloading_rate": "50",
  "weekly_weight_increase_plan": "Week1\nSET1 45%x5\nSET2 50%x5\nSET3 55%x5+\n\nWeek2\nSET1 55%x3\nSET2 60%x3\nSET3 65%x3+\n\nWeek3\nSET1 65%x5\nSET2 70%x3\nSET3 75%x1+\n\nWeek4\nSET1 50%x5\nSET2 55%x5\nSET3 60%x5\n"
}
## 질문
-5/3/1 프로그램
- **운동경험**: 2
- **성별**: 남성
- **운동목표**: 3
- **1RM**:
  - 벤치프레스: 60kg
  - 스쿼트: 90kg
  - 데드리프트: 70kg
  - 오버헤드프레스: 30kg

## 답변
{
  "program": "5/3/1 프로그램",
  "init_weight_rate": "55",
  "increase_rate_week": "10",
  "increase_rate_set": "5",
  "deloading_rate": "30",
  "weekly_weight_increase_plan": "Week1\nSET1 55%x5\nSET2 60%x5\nSET3 65%x5+\n\nWeek2\nSET1 65%x3\nSET2 70%x3\nSET3 75%x3+\n\nWeek3\nSET1 75%x5\nSET2 80%x3\nSET3 85%x1+\n\nWeek4\nSET1 30%x5\nSET2 35%x5\nSET3 40%x5\n"


# Fine-tuning Process

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "EleutherAI/polyglot-ko-1.3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    torch_dtype="auto",
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

In [35]:
import json

print("Rows:", text_df.shape[0])
print("Columns:", text_df.shape[1])

text_df[["text"]].to_json("training_data.jsonl", orient="records", lines=True, force_ascii=False)

Rows: 1000
Columns: 2


In [38]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="training_data.jsonl")["train"]

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [43]:
# Path
fine_tunned_dir = os.path.join(burnfit_dir, "fine_tunned")
path_to_save = os.path.join(fine_tunned_dir, "lora-output-v1")

True

In [44]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir=path_to_save,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=20,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    optim="paged_adamw_8bit"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

<ipython-input-44-bea1592a4555>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [45]:
# Training!!
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,2.248300
40,0.688500
60,0.138400
80,0.078800
100,0.062300
120,0.058700
140,0.056800
160,0.055000
180,0.055200


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during

TrainOutput(global_step=186, training_loss=0.37182582962897515, metrics={'train_runtime': 226.0862, 'train_samples_per_second': 13.269, 'train_steps_per_second': 0.823, 'total_flos': 1.156444204302336e+16, 'train_loss': 0.37182582962897515, 'epoch': 2.96})

# Testing

## Before fine-tuning

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "EleutherAI/polyglot-ko-1.3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True,
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [32]:
prompt = """## 질문
- 5/3/1프로그램
- **운동경험**: 1
- **성별**: 남성
- **운동목표**: 6
- **1RM**:
  - 벤치프레스: 30kg
  - 스쿼트: 30kg
  - 데드리프트: 30kg
  - 오버헤드프레스: 30kg

## 답변
"""

output = generator(
    prompt,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1,
)

print(output[0]["generated_text"])

## 질문
- 5/3/1프로그램
- **운동경험**: 1
- **성별**: 남성
- **운동목표**: 6
- **1RM**:
  - 벤치프레스: 30kg
  - 스쿼트: 30kg
  - 데드리프트: 30kg
  - 오버헤드프레스: 30kg

## 답변
- **질문*** : 안녕하세요. 저는 지금 헬스장에서 운동을 하고 있습니다.제가 하는 운동은 중량을 이용한 근력운동입니다.저는 처음에 중량을 많이 들지 않고 반복횟수만 높여가면서 운동을 했었는데요.지금은 중량을 이용해서 조금씩 횟수를 늘려가며 운동하고 있습니다.근력운동을 시작한지 얼마 안되었지만 운동효과도 좋고 몸도 좋아진것 같아 만족스럽게 생각합니다.질문내용이 궁금한데요..저에게 맞는 운동법이나 제가 추천해줄만한 운동이 있다면 알려주세요~~^^ ---------------------------------------- 답변안녕하십니까? 저는 현재 서울 휘트니스센터의 트레이너로 근무중인 김정환이라고 합니다. 먼저 이렇게 문의 주신점에 감사드립니다. 제가 답변해 드릴 수 있는 부분은 몇가지 안됩니다. 웨이트 트레이닝을 하시려는 분들 중


## after fine-tuning

In [51]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

checkpoint_126 = os.path.join(path_to_save, "checkpoint-126")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b", device_map="auto")
lora_model = PeftModel.from_pretrained(base_model, checkpoint_126)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_126)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [55]:
prompt = """## 질문
- 5/3/1프로그램
- **운동경험**: 3
- **성별**: 남성
- **운동목표**: 2
- **1RM**:
  - 벤치프레스: 60kg
  - 스쿼트: 70kg
  - 데드리프트: 70kg
  - 오버헤드프레스: 70kg

## 답변
"""

output = generator(
    prompt,
    max_new_tokens=260,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1,
)

print(output[0]["generated_text"])

## 질문
- 5/3/1프로그램
- **운동경험**: 3
- **성별**: 남성
- **운동목표**: 2
- **1RM**:
  - 벤치프레스: 60kg
  - 스쿼트: 70kg
  - 데드리프트: 70kg
  - 오버헤드프레스: 70kg

## 답변
{
  "program": "5/3/1프로그램",
  "init_weight_rate": "55",
  "increase_rate_week": "10",
  "increase_rate_set": "5",
  "deloading_rate": "30",
  "weekly_weight_increase_plan": "Week1\nSET1 55%x5\nSET2 65%x5\nSET3 75%x5+\n\nWeek2\nSET1 65%x3\nSET2 75%x3\nSET3 85%x3+\n\nWeek3\nSET1 75%x5\nSET2 85%x3\nSET3 95%x1+\n\nWeek4\nSET1 30%x5\nSET2 40%x5\nSET3 50%x5\n"
}
  "incre
